## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import functions as f

%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [3]:
# manufactoring suppliers 
# 'lrcs', 'klac', ''

In [6]:
f.predictions('oxy')

OXY 5m Interval Timestamp: 2024-11-01 03:53:36 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.475,0.639291,0.490241,0.478045
prob_up,0.125,0.074624,0.120249,0.155564
prob_static,0.625,0.742351,0.635887,0.627175
prob_down,0.25,0.183026,0.243865,0.217261
precision,0.728,0.707602,0.675229,0.587108
recall,0.730924,0.728916,0.738956,0.676707
f1,0.729459,0.718101,0.705657,0.628731
support,"[497.0, 498.0, 498.0]","[497.0, 498.0, 498.0]","[497.0, 498.0, 498.0]","[497.0, 498.0, 498.0]"


OXY 15m Interval Timestamp: 2024-11-01 03:53:49 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.475,0.826872,0.70686,0.430169
prob_up,0.0,0.059503,0.086968,0.205617
prob_static,0.625,0.876337,0.790614,0.592978
prob_down,0.375,0.06416,0.122418,0.201405
precision,0.7,0.689266,0.677249,0.547264
recall,0.768293,0.743902,0.780488,0.670732
f1,0.732558,0.715543,0.725212,0.60274
support,"[164.0, 164.0, 163.0]","[164.0, 164.0, 163.0]","[164.0, 164.0, 163.0]","[164.0, 164.0, 163.0]"


OXY 30m Interval Timestamp: 2024-11-01 03:53:54 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,up,up,static
kelly_1:2.5,0.3,0.554452,0.480956,0.325307
prob_up,0.25,0.681752,0.629254,0.359847
prob_static,0.5,0.301175,0.330316,0.518076
prob_down,0.25,0.017074,0.040429,0.122077
precision,0.708054,0.800725,0.810606,0.54026
recall,0.720137,0.756849,0.732877,0.709898
f1,0.714044,0.778169,0.769784,0.613569
support,"[292.0, 293.0, 292.0]","[292.0, 293.0, 292.0]","[292.0, 293.0, 292.0]","[292.0, 293.0, 292.0]"


OXY 1wk Interval Timestamp: 2024-11-01 03:54:15 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,static,down
kelly_1:2.5,0.3,0.295226,0.549984,0.338498
prob_up,0.25,0.011769,0.029686,0.071747
prob_static,0.25,0.491641,0.67856,0.400755
prob_down,0.5,0.49659,0.291754,0.527498
precision,0.819672,0.857143,0.763636,0.777778
recall,0.862069,0.827586,0.736842,0.724138
f1,0.840336,0.842105,0.75,0.75
support,"[58.0, 57.0, 58.0]","[58.0, 57.0, 58.0]","[58.0, 57.0, 58.0]","[58.0, 57.0, 58.0]"


OXY 1mo Interval Timestamp: 2024-11-01 03:54:18 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,down
kelly_1:2.5,0.3,0.602489,0.653721,0.253399
prob_up,0.0,0.01475,0.076371,0.146104
prob_static,0.5,0.716064,0.752658,0.387182
prob_down,0.5,0.269186,0.170971,0.466714
precision,0.757576,0.8,0.78125,0.619048
recall,0.961538,0.923077,0.961538,0.481481
f1,0.847458,0.857143,0.862069,0.541667
support,"[27.0, 26.0, 27.0]","[27.0, 26.0, 27.0]","[27.0, 26.0, 27.0]","[27.0, 26.0, 27.0]"


### Check Summary Output

In [ ]:
symbol='NVDA'
interval='5m'

# Define Eastern Time Zone
eastern = pytz.timezone('US/Eastern')

# Get current time in Eastern Time Zone
eastern_time = datetime.now(eastern)

# Format the time to include hour, minute, and seconds
time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [27]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')